In [0]:
import os
from google.colab import drive
import cv2
os.chdir("/content")

In [0]:
drive.mount('/content/drive')

In [0]:
!pip install luminoth

# Importing Training Weight from Google Drive.

#### Kindly enter the same name and directory of checkpoint

In [0]:
!cp '/content/drive/My Drive/72209410e475.tar'  '72209410e475.tar'

In [44]:
!lumi checkpoint import 72209410e475.tar


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Checkpoint 72209410e475 imported successfully.


# Prediction From Simple Image

#### Kindly upload some pdf page image For Table Detection. Like the sample image.

In [47]:
# open image file
os.chdir("/content")
img = cv2.imread('test_sample.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# perform transformations on image
b = cv2.distanceTransform(img, distanceType=cv2.DIST_L2, maskSize=5)
g = cv2.distanceTransform(img, distanceType=cv2.DIST_L1, maskSize=5)
r = cv2.distanceTransform(img, distanceType=cv2.DIST_C, maskSize=5)
# merge the transformed channels back to an image
transformed_image = cv2.merge((b, g, r))
cv2.imwrite('test_sample_preprocess.png', transformed_image)

True

**Enter the preprocessed image name and checkpoint name**

In [0]:
predictive=!lumi predict --checkpoint 72209410e475 test_sample_preprocess.png

In [0]:
predictive=eval(predictive[-1])

In [55]:
print ('Total number of tables found: ',len(predictive['objects']))
predictive['objects']

Total number of tables found:  7


[{'bbox': [708, 1509, 2494, 2061], 'label': 'table', 'prob': 0.9486},
 {'bbox': [689, 307, 2546, 2104], 'label': 'table', 'prob': 0.9334},
 {'bbox': [1001, 910, 2485, 1747], 'label': 'table', 'prob': 0.844},
 {'bbox': [763, 316, 2504, 1193], 'label': 'table', 'prob': 0.8342},
 {'bbox': [798, 1604, 1484, 2035], 'label': 'table', 'prob': 0.801},
 {'bbox': [1610, 398, 2465, 2101], 'label': 'table', 'prob': 0.789},
 {'bbox': [661, 350, 2546, 685], 'label': 'table', 'prob': 0.6384}]

**Drawing all the box on the image. you can set the range by change the for loop iterations.**

Image is saved as 'sample_final_output.png'

In [56]:
image = cv2.imread('test_sample.png')
for i in range(len(predictive['objects'])):
  box_cor=predictive['objects'][i]['bbox']
  cv2.rectangle(image, (box_cor[0],box_cor[1]),(box_cor[2],box_cor[3]), (255,0,0), 2)
cv2.imwrite("sample_final_output.png",image)

True

# Prediction from Pdf File

**Run the code pdf to images in local system then Upload the zip file of pdf pages images here.**

This code is made for images for finding tables. In this case we need to first convert the pdf to images. There are multiple libraries exist for this purpose but Unfortunately all the libraries are depends on some kind of software like:



*   ImageMagick
*   pdftoppm (poppler)
*   Ghostscript





These software are written in c++ so I don't think they can be run in colab.

You can't avoid these software dependency. Even Imagemagick relies on Ghostscript for its PDF reading functions. The reason for this is the complexity of the PDF format: a PDF doesn't just contain bitmap information, but mostly vector shapes, transparencies etc. Furthermore it is quite complex to figure out which of these objects appear on which page.

So the correct rendering of a PDF Page is clearly out of scope for a pure Python library.

#### Run the Pdf_To_images.ipynb in local system

Upload the Pdf-Images.zip folder from the local directory.

In [0]:
!mkdir -p Pdf-Images

In [86]:
!unzip "Pdf-Images.zip" -d Pdf-Images

Archive:  Pdf-Images.zip
  inflating: Pdf-Images/out_0.jpg    
  inflating: Pdf-Images/out_1.jpg    
  inflating: Pdf-Images/out_2.jpg    


In [0]:
import glob
Images=glob.glob('Pdf-Images/*.jpg')

In [0]:
!mkdir -p Pdf-Images-Preprocess
for image in Images:
  # open image file
  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # perform transformations on image
  b = cv2.distanceTransform(img, distanceType=cv2.DIST_L2, maskSize=5)
  g = cv2.distanceTransform(img, distanceType=cv2.DIST_L1, maskSize=5)
  r = cv2.distanceTransform(img, distanceType=cv2.DIST_C, maskSize=5)
  # merge the transformed channels back to an image
  transformed_image = cv2.merge((b, g, r))
  cv2.imwrite(image.replace('Pdf-Images','Pdf-Images-Preprocess'), transformed_image)

**Enter the preprocessed image name and checkpoint name**

**Predictions on all images.**

Image is saved as 'sample_final_output.png'

In [0]:
Images_preprocessed=glob.glob('Pdf-Images-Preprocess/*.jpg')

In [0]:
from luminoth import Detector, read_image, vis_objects
!mkdir -p Pdf_final_images
detector = Detector(checkpoint='72209410e475')

In [0]:
for l in range(len(Images_preprocessed)):
  image = read_image(Images_preprocessed[l])

  # Returns a dictionary with the detections.
  objects = detector.predict(image)
  
  image_save=read_image(Images[l])
  
  vis_objects(image_save, objects).save(Images_preprocessed[l].replace('Pdf-Images-Preprocess','Pdf_final_images'))
  
  

# Saving the Images as PDF File

In [106]:
!pip install fpdf

  Stored in directory: /root/.cache/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf


In [0]:
from os import listdir
from fpdf import FPDF

In [110]:
path = "Pdf_final_images/" # get the path of images

imagelist = listdir(path) # get list of all images

pdf = FPDF('P','mm','A4') # create an A4-size pdf document 

x,y,w,h = 0,0,200,250

for image in imagelist:

    pdf.add_page()
    pdf.image(path+image,x,y,w,h)

pdf.output("pdf_with_table.pdf","F")

''